Initialize channel configuration and computational mesh for three-body bound state calculation

In [1]:
include("../general_modules/channels.jl")
include("../general_modules/mesh.jl")
using .channels
using .mesh

fermion=true; Jtot = 0.5; T = 0.5; Parity=1
lmax=2; lmin=0; λmax=0; λmin=0; s1=0.5; s2=0.5; s3=0.5; t1=0.5; t2=0.5; t3=0.5; MT=-0.5 # -0.5 for 3H
nθ=12; nx=30; ny=30; xmax=20; ymax=20; alpha=1

α= α3b(fermion,Jtot,T,Parity,lmax,lmin,λmax,λmin,s1,s2,s3,t1,t2,t3,MT)
grid= initialmesh(nθ,nx,ny,Float64(xmax),Float64(ymax),Float64(alpha));

nothing

For J=0.5 T=0.5 parity=1 Number of channels: 3
---The coupling coefficients are
 a3b |( l ( s1 s2 ) s12 ) J12 ( λ s3 ) J3 ,   J; ( t1 t2 ) T12 , t3 , T >
   1 |( 0 (0.5 0.5) 0.0)  0.0 ( 0 0.5) 0.5, 0.5; (0.5 0.5) 1.0, 0.5, 0.5 > 
   2 |( 0 (0.5 0.5) 1.0)  1.0 ( 0 0.5) 0.5, 0.5; (0.5 0.5) 0.0, 0.5, 0.5 > 
   3 |( 2 (0.5 0.5) 1.0)  1.0 ( 0 0.5) 0.5, 0.5; (0.5 0.5) 0.0, 0.5, 0.5 > 
scaling factor for x: 0.18859695545615252


Calculate deuteron bound state using nuclear potential and analyze channel composition

In [2]:
include("twobody.jl")
using .twobodybound

potname="AV18"
e2b, ψ =bound2b(grid, potname); 

nothing


Two-body channel configuration:
  Total angular momentum J = 1.0
  Parity = +
  Number of channels: 2
    Channel 1: 3S₁ (l=0, s=1.0)
    Channel 2: 3D₁ (l=2, s=1.0)

           TWO-BODY BOUND STATE ANALYSIS

Bound State #1:
  Binding Energy: -2.223939 MeV
  Total J^π = 1⁺

  Channel Composition:
    Channel 1: 3S₁ (l=0, s=1.0) - 94.24%
    Channel 2: 3D₁ (l=2, s=1.0) - 5.76%

  D-state Probability: 5.762%
  S-state Probability: 94.238%

SUMMARY: Found 1 bound state(s)
Binding energies (MeV): [-2.223939]


In [ ]:
computing the 3H bound state by using Faddeev Equations

In [ ]:
include("threebodybound.jl")
using .threebodybound

ThreeBody_Bound(α, grid, potname,e2b); 

nothing


         THREE-BODY BOUND STATE ANALYSIS
Two-body threshold energy: -2.223939 MeV
Searching for three-body bound states...

Three-body Bound State #1:
  Binding Energy: -6.830823 MeV
  Energy gain from 2-body: -4.606884 MeV
  Wave function norm: 1.0
  Eigenvalue energy: -6.830823 MeV
  ⟨ψ|H|ψ⟩ energy: -6.830823 MeV
  Energy difference: 0.0 MeV
  ✓ Energy verification: PASSED

SUMMARY: Found 1 three-body bound state(s)
Three-body binding energies (MeV): [-6.830823]


1-element Vector{Any}:
 (-6.8308231176225735 + 2.0609295520168658e-13im, ComplexF64[6.913375601269424e-7 + 9.582060215814273e-21im, 3.4942649522983103e-6 + 6.003346013664854e-20im, 1.0299345344734428e-5 + 1.2086101366134798e-19im, 2.7283926182419627e-5 + 2.6808592780598606e-19im, 7.007680701752538e-5 + 3.324604317973129e-19im, 0.00016670241781706392 + 1.0757318430129614e-19im, 0.0003356625888087054 - 2.608861477543245e-19im, 0.0005297814575777969 - 3.6253010142484055e-19im, 0.0006543179255370353 - 6.437450399132683e-20im, 0.0006709593210072243 + 2.608861477543245e-19im  …  3.12303747060228e-6 + 1.0338301819034753e-18im, 4.0261660968267875e-6 - 7.072725109573408e-20im, 4.441358391291862e-6 - 1.2089013041890985e-18im, 4.4471348296727155e-6 + 2.799840937369488e-18im, 4.076453739406441e-6 - 1.2033691202523438e-18im, 3.4839116472038895e-6 + 4.249723115052435e-18im, 2.7255177650807667e-6 - 6.951890565691662e-19im, 1.978876687261045e-6 + 1.1950046698982076e-18im, 1.2470741509073435e-6 - 4.395